# Task 3: 进阶上分方向

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['font.sans-serif'] = ["WenQuanYi Micro Hei",'SimHei']
plt.rcParams['axes.unicode_minus'] = False

base_path=Path("data")
train_length=55392

In [16]:
electricity_price=pd.read_csv(base_path/"electricity_price_parsed.csv",parse_dates=["timestamp"],index_col=0)
electricity_price.columns=["demand","price"]
sample_submit=pd.read_csv(base_path/"sample_submit.csv")

train_data=electricity_price.copy(deep=True)
electricity_price.head()

,demand,price
timestamp,,
2021-12-01 00:15:00,40334.18,350.80
2021-12-01 00:30:00,40523.15,350.80
2021-12-01 00:45:00,40374.74,350.80
2021-12-01 01:00:00,40111.55,350.80
2021-12-01 01:15:00,40067.50,348.93


## 时间序列法

首先，先根据Task2的讨论，进行特征提取

In [17]:
# 将电价数据的时间索引信息添加到训练数据中

# 提取时间索引的小时信息，并添加到训练数据中，创建 "hour" 列
train_data["hour"] = electricity_price.index.hour

# 提取时间索引的日期信息，并添加到训练数据中，创建 "day" 列
train_data["day"] = electricity_price.index.day

# 提取时间索引的月份信息，并添加到训练数据中，创建 "month" 列
train_data["month"] = electricity_price.index.month

# 提取时间索引的年份信息，并添加到训练数据中，创建 "year" 列
train_data["year"] = electricity_price.index.year

# 提取时间索引的星期信息，并添加到训练数据中，创建 "weekday" 列
train_data["weekday"] = electricity_price.index.weekday

# 根据月份信息，判断是否为风季（1-5月和9-12月），创建布尔型 "is_windy_season" 列
train_data["is_windy_season"] = electricity_price.index.month.isin([1, 2, 3, 4, 5, 9, 10, 11, 12])

# 根据小时信息，判断是否为低谷时段（10-15点），创建布尔型 "is_valley" 列
train_data["is_valley"] = electricity_price.index.hour.isin([10, 11, 12, 13, 14, 15])

# 提取时间索引的季度信息，并添加到训练数据中，创建 "quarter" 列
train_data["quarter"] = electricity_price.index.quarter

# 对时间特征进行独热编码（One-Hot Encoding），删除第一列以避免多重共线性
train_data = pd.get_dummies(
    data=train_data,        # 需要进行独热编码的 DataFrame
    columns=["hour", "day", "month", "year", "weekday"],  # 需要独热编码的列
    drop_first=True         # 删除第一列以避免多重共线性
)

train_data

,demand,price,is_windy_season,is_valley,quarter,hour_1,hour_2,hour_3,hour_4,hour_5,...,month_12,year_2022,year_2023,year_2024,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 00:15:00,40334.18,350.80,True,False,4,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2021-12-01 00:30:00,40523.15,350.80,True,False,4,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2021-12-01 00:45:00,40374.74,350.80,True,False,4,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2021-12-01 01:00:00,40111.55,350.80,True,False,4,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2021-12-01 01:15:00,40067.50,348.93,True,False,4,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-18 23:00:00,35861.48,NaN,True,False,2,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
2024-04-18 23:15:00,35862.24,NaN,True,False,2,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
2024-04-18 23:30:00,35005.55,NaN,True,False,2,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False


In [18]:
# 构造节假日虚拟变量，这里个人在官方代码的基础上增加国庆节假期

def generate_holiday_dates(start_dates, duration):
    holidays = []  # 初始化一个空列表，用于存储节假日日期
    for start_date in start_dates:  # 遍历每个节假日的开始日期
        # 生成从 start_date 开始的日期范围，持续时间为 duration 天
        holidays.extend(pd.date_range(start=start_date, periods=duration).tolist())
    return holidays  # 返回所有节假日日期的列表

spring_festival_start_dates = ["2022-01-31", "2023-01-21", "2024-02-10"]
labor_start_dates = ["2022-04-30", "2023-04-29"]
national_start_dates = ['2022-10-01', '2023-10-01']
spring_festivals = generate_holiday_dates(spring_festival_start_dates, 7)
labor = generate_holiday_dates(labor_start_dates, 5)
national = generate_holiday_dates(national_start_dates, 7)

# 判断训练数据的索引是否在春节日期列表中，生成布尔型列 "is_spring_festival"
train_data["is_spring_festival"] = train_data.index.isin(spring_festivals)

# 判断训练数据的索引是否在劳动节日期列表中，生成布尔型列 "is_labor"
train_data["is_labor"] = train_data.index.isin(labor)

# 判断训练数据的索引是否在国庆节节日期列表中，生成布尔型列 "is_national"
train_data["is_national"] = train_data.index.isin(national)

此外，我们还可以用滑动窗口构造特征：

In [19]:
def cal_range(x): # 极差
    return x.max() - x.min()


def increase_num(x): # 发生增长的次数
    return (x.diff() > 0).sum()


def decrease_num(x): # 发生下降的次数
    return (x.diff() < 0).sum()


def increase_mean(x): # 上升序列均值
    diff = x.diff()
    return diff[diff > 0].mean()


def decrease_mean(x): # 下降序列均值
    diff = x.diff()
    return diff[diff < 0].abs().mean()


def increase_std(x): # 上升序列标准差
    diff = x.diff()
    return diff[diff > 0].std()


def decrease_std(x): # 下降序列标准差
    diff = x.diff()
    return diff[diff < 0].std()


In [20]:
from tqdm import tqdm  # 导入 tqdm 库用于显示进度条

# 定义滚动窗口大小的列表
window_sizes = [4, 12, 24]

# 遍历每个窗口大小
with tqdm(window_sizes) as pbar:
    for window_size in pbar:
        # 定义要应用的聚合函数列表
        functions = ["mean", "std", "min", "max", cal_range, increase_num,
                     decrease_num, increase_mean, decrease_mean, increase_std, decrease_std]

        # 遍历每个聚合函数
        for func in functions:
            # 获取函数名称，如果是字符串则直接使用，否则使用函数的 __name__ 属性
            func_name = func if type(func) == str else func.__name__

            # 生成新列名，格式为 demand_rolling_{window_size}_{func_name}
            column_name = f"demand_rolling_{window_size}_{func_name}"

            # 计算滚动窗口的聚合值，并将结果添加到 train_data 中
            train_data[column_name] = train_data["demand"].rolling(
                window=window_size,        # 滚动窗口大小
                min_periods=window_size//2,  # 最小观测值数
                closed="left"         # 滚动窗口在左侧闭合
            ).agg(func)              # 应用聚合函数

            pbar.set_postfix({"window_size": window_size, "func": func_name})

# 跑了7分钟。。

100%|██████████| 3/3 [07:30<00:00, 150.26s/it, window_size=24, func=decrease_std] 


此外，还可以加入经典计量学的特征，如滞后、差分等

In [21]:
# 一阶滞后
train_data["demand_shift_1"] = train_data["demand"].shift(1)

# 一阶差分
train_data["demand_diff_1"] = train_data["demand"].diff(1)

# 算术增长率
train_data["demand_pct_1"] = train_data["demand"].pct_change(1)

In [22]:
# 填充因为滞后、差分等形成的NaN：

# 从 train_data 中创建训练集和测试集特征数据 (X) 和目标数据 (y)

# 创建训练集特征数据 X_train
# 1. 从 train_data 中选择前 train_length 行，去除 "price" 列
# 2. 使用 bfill 方法向后填充缺失值
# 3. 使用 ffill 方法向前填充缺失值
X_train = train_data.iloc[:train_length].drop(columns=["price"]).bfill().ffill()

# 创建测试集特征数据 X_test
X_test = train_data.iloc[train_length:].drop(columns=["price"]).bfill().ffill()

# 创建训练集目标数据 y_train
y_train = train_data.iloc[:train_length][["price"]]

模型回归：

In [24]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

# 创建 LGBMRegressor 模型对象，设置参数
# num_leaves：树的叶子数，控制模型复杂度
# n_estimators：迭代次数，即树的数量
# verbose：日志信息级别，-1 表示不输出日志信息
lgb_model = LGBMRegressor(num_leaves=2**5-1, n_estimators=300, verbose=-1)

# 创建线性回归模型对象
linear_model = LinearRegression()

# 使用训练集数据训练 LGBMRegressor 模型
# X_train：训练集特征数据
# y_train：训练集目标数据
lgb_model.fit(X_train, y_train)

# 使用训练集数据中的 "demand" 特征训练线性回归模型
# X_train[["demand"]]：训练集特征数据中仅包含 "demand" 列
# y_train：训练集目标数据
linear_model.fit(X_train[["demand"]], y_train)

# 使用训练好的 LGBMRegressor 模型预测测试集特征数据
# X_test：测试集特征数据
# 返回预测的目标值
lgb_pred = lgb_model.predict(X_test)

# 使用训练好的线性回归模型预测测试集特征数据中的 "demand" 列
# X_test[["demand"]]：测试集特征数据中仅包含 "demand" 列
# 返回预测的目标值，并将结果展平为一维数组
linear_pred = linear_model.predict(X_test[["demand"]]).flatten()

In [25]:
# 最后的修正：
# 简单求均值
y_pred = (lgb_pred+linear_pred)/2
y_pred *= 0.95  # 进行少量修正

In [26]:
# 保存
sample_submit["clearing price (CNY/MWh)"] = y_pred
sample_submit.to_csv("submit.csv", index=False)
sample_submit.head()

,day,time,clearing price (CNY/MWh)
0,2023/7/1,0:15,391.704814
1,2023/7/1,0:30,386.056237
2,2023/7/1,0:45,379.044964
3,2023/7/1,1:00,376.880418
4,2023/7/1,1:15,382.486849
